In [ ]:
# (C) Copyright Aaron Goldberg, 2022.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [ ]:
# Run parameter optimization for each trial individually
import numpy as np
import time

num_shots=1000000
num_jobs=1000
def read_data_from_file(mode_pair, file_counter):
    return np.loadtxt(f'TMSV_data/{mode_pair}_{file_counter}.csv', delimiter=',',dtype ='int')#read back as integers

pair_ID='04'

import scipy
import scipy.special
from scipy.optimize import minimize 
from scipy.optimize import LinearConstraint 
from scipy import stats


upper_state_cutoff=100 # This does need to change because even a large |nn> has a chance of losing enough photons to look like 0 to 9 photons remaining, so it adds a tiny bit to the overall probability of detecting that many photons. At least three sig figs remain the same when going from 20 to 25, so 25 should be sufficient
r=1.0

    
def total_error04KL5(params):
    # Find the KL divergence in the probability distribution as a function of the guessed error value from the ideal case
    eta1=params[0]
    eta2=params[1]
    eta3=params[2]
    v1=params[3]
    v2=params[4]
    state_coefficients_squared=[np.tanh(eta3*r)**(2*n)/np.cosh(eta3*r)**2 for n in range(upper_state_cutoff)]
    my_probabilities=np.array([[eta1**(2*M)*eta2**(2*N)*(1-eta1**2)**(-M)*(1-eta2**2)**(-N)
                                *sum([state_coefficients_squared[m]*scipy.special.binom(m, M)*scipy.special.binom(m, N)*((1-eta1**2)*(1-eta2**2))**(m) for m in range(max(M,N),upper_state_cutoff)])
                                     for N in range(maxval+1)] for M in range(maxval+1)])
    my_darkened_probs=np.array([[np.exp(-v1-v2)*np.sum([[v1**(m-k)*v2**(n-l)*my_probabilities[k,l]/scipy.special.factorial(m-k)/scipy.special.factorial(n-l) for k in range(m+1)]for l in range(n+1)]) 
                                 for n in range(maxval+1)]for m in range(maxval+1)])
    my_probabilities=my_darkened_probs/my_darkened_probs.sum()
    KL_divergence=scipy.stats.entropy(counts04.flatten(),qk=my_probabilities.flatten()) # Second argument is for the "theory" - we want the ideal "theory" for this experiment
    return(KL_divergence)

my_bounds=((0.1,0.9),(0.1,0.9),(0.3,3.0),(0.001,0.2),(0.001,0.2))
my_guess=[0.65,0.65,0.75,0.02,0.02]
all_res=np.zeros((num_jobs,5))



start = time.time()
for job_counter in range(num_jobs):
    counts04= read_data_from_file(pair_ID, job_counter)#it seems as though measuring KL divergence may be slightly faster with unnormalized ints instead of probabilities, but yet to be seen definitively. They yield same results.
    maxval=len(counts04)-1
    all_res[job_counter,:]=scipy.optimize.minimize(total_error04KL5,x0=my_guess, bounds=my_bounds).x
end = time.time()
print('timing:',end - start)

In [ ]:
# Do stats:
means=np.mean(all_res,axis=0)
print("eta1, eta2, effective r, dark count rate 1, dark count rate 2 mean values: ", means)
covariances=np.cov(all_res.T)
print("eta1, eta2, effective r, dark count rate 1, dark count rate 2 covariance values: ", covariances)

# Total energy in TMSV should be 2 sinh(r)^2, with r=1, if we have the correct hbar conventions
nbar=np.sinh(1.0)**2 # Total energy /2, giving the average energy you could give each mode for classical comparison
QFI_inverse_classical=1.0/(4*nbar*num_jobs*num_shots) # Need to include number of trials here
print("min variance for classical state with similar energy, repeated 10^9 times: ",QFI_inverse_classical)
QFI_inverse_Fock=(1-np.mean(all_res,axis=0)**2)/(4*nbar*num_jobs*num_shots) # Need to include number of trials here
print("min variance for Fock state with similar energy, repeated 10^9 times: ",QFI_inverse_Fock)

import matplotlib.pyplot as plt

# The histogram of the data
plt.hist(all_res[:,0:2], 50, density=True, alpha=0.75,label=[pair_ID[0],pair_ID[1]])
plt.legend(loc='upper right')
plt.xlabel(r'$\hat{\eta}$')
plt.ylabel('Frequency (per 1000 trials)')
plt.plot()
plt.savefig('estimators04_etas5par.eps')
plt.show()

# The histogram of the data
plt.hist(all_res[:,2], 50, density=True, alpha=0.75)
#plt.legend(loc='upper right')
plt.xlabel(r'$\hat{r}$')
plt.ylabel('Frequency (per 1000 trials)')
plt.plot()
plt.savefig('estimators04_r5par.eps')
plt.show()

print(all_res)
np.savetxt('MLE_estimates04_5par.csv', all_res, delimiter=',')
new_means=np.mean(np.sqrt(all_res),axis=0)**2 #Estimating a function of the parameter is the same as the function of the estimate (functional equivariance). If MLE converges, the averages of these different estimates will be consistent
print('original means: ',means,', new means:',new_means)

In [ ]:
######### What if we make artificial data? 
# Combine sets of the 10^6 shots from each trial into some bigger groups, do MLE with each of them, see what happens
import numpy as np
import time

num_shots=1000000
num_jobs=1000
num_combined_groups=10 # must be a factor of num_jobs
shots_per_group=int(num_shots*num_jobs/num_combined_groups)
jobs_per_group=int(num_jobs/num_combined_groups)
def read_data_from_file(mode_pair, file_counter):
    return np.loadtxt(f'TMSV_data/{mode_pair}_{file_counter}.csv', delimiter=',',dtype ='int')#read back as integers

pair_ID='04'

#Now do KL divergence but normalize the probability distributions to only include terms that can be measured, as per https://iopscience.iop.org/article/10.1088/1367-2630/10/4/043022/pdf
#This normalization won't change much because the probability of not being registered by the PNRDs is tiny
#Really, we have the probability of measuring more than nmax photons just contributing to the probability for nmax
#This time, 3 parameters: loss in the pump beam, if it is a coherent state, does r-> eta0 r
import scipy
import scipy.special
from scipy.optimize import minimize 
from scipy.optimize import LinearConstraint 
from scipy import stats


upper_state_cutoff=100 # This does need to change because even a large |nn> has a chance of losing enough photons to look like 0 to 9 photons remaining, so it adds a tiny bit to the overall probability of detecting that many photons. At least three sig figs remain the same when going from 20 to 25, so 25 should be sufficient
r=1.0
    
def total_error04KL5(params):
    # Find the KL divergence in the probability distribution as a function of the guessed error value from the ideal case
    eta1=params[0]
    eta2=params[1]
    eta3=params[2]
    v1=params[3]
    v2=params[4]
    state_coefficients_squared=[np.tanh(eta3*r)**(2*n)/np.cosh(eta3*r)**2 for n in range(upper_state_cutoff)]
    my_probabilities=np.array([[eta1**(2*M)*eta2**(2*N)*(1-eta1**2)**(-M)*(1-eta2**2)**(-N)
                                *sum([state_coefficients_squared[m]*scipy.special.binom(m, M)*scipy.special.binom(m, N)*((1-eta1**2)*(1-eta2**2))**(m) for m in range(max(M,N),upper_state_cutoff)])
                                     for N in range(maxval+1)] for M in range(maxval+1)])
    my_darkened_probs=np.array([[np.exp(-v1-v2)*np.sum([[v1**(m-k)*v2**(n-l)*my_probabilities[k,l]/scipy.special.factorial(m-k)/scipy.special.factorial(n-l) for k in range(m+1)]for l in range(n+1)]) 
                                 for n in range(maxval+1)]for m in range(maxval+1)])
    my_probabilities=my_darkened_probs/my_darkened_probs.sum()
    KL_divergence=scipy.stats.entropy(counts04.flatten(),qk=my_probabilities.flatten()) # Second argument is for the "theory" - we want the ideal "theory" for this experiment
    return(KL_divergence)

my_bounds=((0.1,0.9),(0.1,0.9),(0.3,3.0),(0.001,0.2),(0.001,0.2))
my_guess=[0.65,0.65,0.75,0.02,0.02]
all_res=np.zeros((num_combined_groups,5))

combined_group_counter=0

start = time.time()
for job_counter in range(0,num_jobs,jobs_per_group):
    counts04=read_data_from_file(pair_ID, job_counter)
    for group_counter in range(jobs_per_group-1):
        job_counter=job_counter+1
        counts04=counts04+read_data_from_file(pair_ID, job_counter)
    maxval=len(counts04)-1
    all_res[combined_group_counter,:]=scipy.optimize.minimize(total_error04KL5,x0=my_guess, bounds=my_bounds).x
    combined_group_counter=combined_group_counter+1
end = time.time()
print('timing:',end - start)


In [ ]:
# Do stats:
means=np.mean(all_res,axis=0)
print("eta1, eta2, effective r, dark count rate 1, dark count rate 2 mean values: ", means)
covariances=np.cov(all_res.T)
print("eta1, eta2, effective r, dark count rate 1, dark count rate 2 covariance values: ", covariances)

# Total energy in TMSV should be 2 sinh(r)^2, with r=1, if we have the correct hbar conventions
nbar=np.sinh(1.0)**2 # Total energy /2, giving the average energy you could give each mode for classical comparison
QFI_inverse_classical=1.0/(4*nbar*num_jobs*num_shots) # Need to include number of trials here
print("min variance for classical state with similar energy, repeated 10^9 times: ",QFI_inverse_classical)
QFI_inverse_Fock=(1-np.mean(all_res,axis=0)**2)/(4*nbar*num_jobs*num_shots) # Need to include number of trials here
print("min variance for Fock state with similar energy, repeated 10^9 times: ",QFI_inverse_Fock)

import matplotlib.pyplot as plt

# The histogram of the data
plt.hist(all_res[:,0:2], 50, density=True, alpha=0.75,label=[pair_ID[0],pair_ID[1]])
plt.legend(loc='upper right')
plt.xlabel(r'$\hat{\eta}$')
plt.ylabel('Frequency (per 10 trials)')
plt.plot()
#plt.savefig('estimators04_etas_10groups.eps')
plt.show()

# The histogram of the data
plt.hist(all_res[:,2], 50, density=True, alpha=0.75)
#plt.legend(loc='upper right')
plt.xlabel(r'$\hat{r}$')
plt.ylabel('Frequency (per 10 trials)')
plt.plot()
#plt.savefig('estimators04_r_10groups.eps')
plt.show()

print(all_res)
#np.savetxt('MLE_estimates04_10groups.csv', all_res, delimiter=',')
new_means=np.mean(np.sqrt(all_res),axis=0)**2 #Estimating a function of the parameter is the same as the function of the estimate (functional equivariance). If MLE converges, the averages of these different estimates will be consistent
print('original means: ',means,', new means:',new_means)